In [1]:
import os
import numpy as np
import scipy.io as sio

from paican.paican import PAICAN

In [2]:
path = 'data/parliament/'

A = sio.mmread(os.path.join(path, 'A.mtx'))
X = sio.mmread(os.path.join(path, 'X.mtx'))
z = np.load(os.path.join(path, 'z.npy'))
K = len(np.unique(z))

In [3]:
paican = PAICAN(A, X, K, verbose=True)
z_pr, ca_pr, cx_pr = paican.fit_predict()

iter   0, ELBO: -1751.73865
iter   1, ELBO: -1590.77551
iter   2, ELBO: -1579.56189
iter   3, ELBO: -1578.55005
iter   4, ELBO: -1578.30286
iter   5, ELBO: -1578.21875
iter   6, ELBO: -1578.15271
iter   7, ELBO: -1578.12732
iter   8, ELBO: -1578.08594
iter   9, ELBO: -1578.05005
iter  10, ELBO: -1577.97253
iter  11, ELBO: -1577.84399
iter  12, ELBO: -1577.62976
iter  13, ELBO: -1577.29810
iter  14, ELBO: -1576.77087
iter  15, ELBO: -1576.05811
iter  16, ELBO: -1575.55151
iter  17, ELBO: -1575.44727
iter  18, ELBO: -1575.42542
iter  19, ELBO: -1575.38208
iter  20, ELBO: -1575.35608
iter  21, ELBO: -1575.30725
iter  22, ELBO: -1575.29565
iter  23, ELBO: -1575.25049
iter  24, ELBO: -1575.21362
iter  25, ELBO: -1575.18054
iter  26, ELBO: -1575.16113
iter  27, ELBO: -1575.15735
iter  28, ELBO: -1575.15613
iter  29, ELBO: -1575.12366
iter  30, ELBO: -1575.11975
iter  31, ELBO: -1575.12085
iter  32, ELBO: -1575.12268
iter  33, ELBO: -1575.12781
iter  34, ELBO: -1575.12561
iter  35, ELBO: -157